In [ ]:
import os
import time
import random
import pickle
import argparse

import numpy as np
import scipy.sparse as sp
import scipy.signal as sig
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Radar
with open(os.path.join("waveforms/resp/train_radar_all/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    train_pred = data['pred']
    train_gt = data['gt']
    train_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(train_pred), len(train_gt), len(train_participant_task_chunk_id_list))
with open(os.path.join("waveforms/resp/val_radar_all/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    val_pred = data['pred']
    val_gt = data['gt']
    val_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(val_pred), len(val_gt), len(val_participant_task_chunk_id_list))
with open(os.path.join("waveforms/resp/test_radar_all/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    test_pred = data['pred']
    test_gt = data['gt']
    test_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(test_pred), len(test_gt), len(test_participant_task_chunk_id_list))
radar_pred = np.concatenate([train_pred, val_pred, test_pred], axis=0)
radar_gt = np.concatenate([train_gt, val_gt, test_gt], axis=0)
radar_participant_task_chunk_id_list = train_participant_task_chunk_id_list + val_participant_task_chunk_id_list + test_participant_task_chunk_id_list
print(len(radar_pred), len(radar_gt), len(radar_participant_task_chunk_id_list))
print()

# Thermal Camera
with open(os.path.join("waveforms/resp/train_fusion/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    train_pred = data['pred']
    train_gt = data['gt']
    train_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(train_pred), len(train_gt), len(train_participant_task_chunk_id_list))
with open(os.path.join("waveforms/resp/val_fusion/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    val_pred = data['pred']
    val_gt = data['gt']
    val_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(val_pred), len(val_gt), len(val_participant_task_chunk_id_list))
with open(os.path.join("waveforms/resp/test_fusion/", "pred.pickle"), 'rb') as f:
    data = pickle.load(f)
    test_pred = data['pred']
    test_gt = data['gt']
    test_participant_task_chunk_id_list = data['participant_task_chunk_id_list']
print(len(test_pred), len(test_gt), len(test_participant_task_chunk_id_list))
therm_pred = np.concatenate([train_pred, val_pred, test_pred], axis=0)
therm_gt = np.concatenate([train_gt, val_gt, test_gt], axis=0)
therm_participant_task_chunk_id_list = train_participant_task_chunk_id_list + val_participant_task_chunk_id_list + test_participant_task_chunk_id_list
print(len(therm_pred), len(therm_gt), len(therm_participant_task_chunk_id_list))

In [ ]:
pred_waveform = {participant_task: [] for participant_task, chunk in radar_participant_task_chunk_id_list}
gt_waveform = {participant_task: [] for participant_task, chunk in radar_participant_task_chunk_id_list}
for pred, gt, participant_task_chunk_id in zip(radar_pred, radar_gt, radar_participant_task_chunk_id_list):
    # print(pred.shape, gt.shape, participant_task_chunk_id)
    participant_task = participant_task_chunk_id[0]
    chunk_id = participant_task_chunk_id[1]
    pred_waveform[participant_task].extend(pred.tolist())
    gt_waveform[participant_task].extend(gt.tolist())
for participant_task in pred_waveform:
    pred_waveform[participant_task] = np.array(pred_waveform[participant_task])
    gt_waveform[participant_task] = np.array(gt_waveform[participant_task])
    # print(participant_task, pred_waveform[participant_task].shape, gt_waveform[participant_task].shape)
radar_waveforms = {}
for participant_task in pred_waveform:
    radar_waveforms[participant_task] = {
        'pred': pred_waveform[participant_task],
        'gt': gt_waveform[participant_task]
    }
print(radar_waveforms.keys())


pred_waveform = {participant_task: [] for participant_task, chunk in therm_participant_task_chunk_id_list}
gt_waveform = {participant_task: [] for participant_task, chunk in therm_participant_task_chunk_id_list}
for pred, gt, participant_task_chunk_id in zip(therm_pred, therm_gt, therm_participant_task_chunk_id_list):
    # print(pred.shape, gt.shape, participant_task_chunk_id)
    participant_task = participant_task_chunk_id[0]
    chunk_id = participant_task_chunk_id[1]
    pred_waveform[participant_task].extend(pred.tolist())
    gt_waveform[participant_task].extend(gt.tolist())
for participant_task in pred_waveform:
    pred_waveform[participant_task] = np.array(pred_waveform[participant_task])
    gt_waveform[participant_task] = np.array(gt_waveform[participant_task])
    # print(participant_task, pred_waveform[participant_task].shape, gt_waveform[participant_task].shape)
therm_waveforms = {}
for participant_task in pred_waveform:
    therm_waveforms[participant_task] = {
        'pred': pred_waveform[participant_task],
        'gt': gt_waveform[participant_task]
    }
print(therm_waveforms.keys())

In [ ]:
len(therm_waveforms.keys()), len(radar_waveforms.keys())

In [ ]:
# Combine both dicts 
combined_waveforms = {}
for participant_task in therm_waveforms:
    if participant_task in radar_waveforms:
        combined_waveforms[participant_task] = {
            'radar': radar_waveforms[participant_task]['pred'],
            'thermal': therm_waveforms[participant_task]['pred'],
            'gt': radar_waveforms[participant_task]['gt']
        }
        assert np.array_equal(radar_waveforms[participant_task]['gt'], therm_waveforms[participant_task]['gt']), f"GT mismatch for {participant_task}"
    else:
        print(f"Participant task {participant_task} not found in radar waveforms")

In [ ]:
combined_waveforms[participant_task]['radar'].shape, combined_waveforms[participant_task]['thermal'].shape, combined_waveforms[participant_task]['gt'].shape

In [ ]:
# Save as npy array. Split the waveform into 20 second chunks
fs = 15
time = 120
chunk_time = 20
root_folder = "combined_respiratory_waveforms"
os.makedirs(root_folder, exist_ok=True)
for participant_task in combined_waveforms:
    save_folder = os.path.join(root_folder, participant_task)
    os.makedirs(save_folder, exist_ok=True)
    radar_waveform = combined_waveforms[participant_task]['radar']
    therm_waveform = combined_waveforms[participant_task]['thermal']
    gt_waveform = combined_waveforms[participant_task]['gt']
    num_chunks = int(np.ceil(radar_waveform.shape[0] / (fs * chunk_time)))
    print(f"Participant task {participant_task} has {num_chunks} chunks")
    radar_chunks = np.split(radar_waveform, num_chunks)
    therm_chunks = np.split(therm_waveform, num_chunks)
    gt_chunks = np.split(gt_waveform, num_chunks)
    for i in range(num_chunks):
        radar_wave = radar_chunks[i]
        therm_wave = therm_chunks[i]
        gt_wave = gt_chunks[i]
        np.save(os.path.join(save_folder, f"radar_waveform_{i}.npy"), radar_wave)
        np.save(os.path.join(save_folder, f"thermal_waveform_{i}.npy"), therm_wave)
        np.save(os.path.join(save_folder, f"gt_waveform_{i}.npy"), gt_wave)